
# Data Preparation Notebook

This Jupyter notebook contains steps for preparing the dataset for further analysis and modeling. 
It includes data cleaning, handling missing data, and (pre-)processing data.

## Steps Included:
1. Converting Date and Time Columns
2. Transforming Numeric Data
3. Handling Missing Data
4. Cleaning String Data
5. Data Quality Assessment

Let's start by loading the dataset and necessary libraries.


In [ ]:

import pandas as pd

# Load the dataset
file_path = 'combined_data.csv'  # Update the path if needed
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()



## Convert Date and Time Columns

We'll convert columns with dates and times into a standard datetime format.


In [ ]:

# Converting columns to datetime format
date_columns = ['DatumFileBegin', 'DatumFileEind']
time_columns = ['TijdFileBegin', 'TijdFileEind']

for col in date_columns:
    data[col] = pd.to_datetime(data[col])

for col in time_columns:
    data[col] = pd.to_datetime(data[col], format='%H:%M:%S').dt.time

# Display the updated dataset
data.head()



## Transform Numeric Data

Transform columns with numeric data stored as strings into numeric formats.


In [ ]:

# Convert numeric data stored as strings to numeric format
numeric_columns = ['FileZwaarte', 'GemLengte']

for col in numeric_columns:
    data[col] = data[col].str.replace(',', '').astype(float)

# Display the updated dataset
data.head()



## Handling Missing Data

Identifying and addressing missing data in the dataset.


In [ ]:

# Check for missing values
missing_data = data.isnull().sum()
missing_data[missing_data > 0]



In [ ]:
# Drop rows with any missing values
data = data.dropna()

# Verify the operation by checking for missing values again
missing_data_after = data.isnull().sum()
missing_data_after[missing_data_after > 0]


## Cleaning String Data

Trimming unnecessary whitespaces and handling new lines in string columns.


In [ ]:
# Cleaning string columns
string_columns = data.select_dtypes(include='object').columns

for col in string_columns:
    data[col] = data[col].astype(str).str.strip()

# Display the updated dataset
data.head()


## Removing Square Brackets in 'OorzaakVerloop' Column

In the `OorzaakVerloop` column, each entry is enclosed within square brackets, which are not necessary for our analysis and may interfere with certain data processing tasks. To clean this data, we will remove the square brackets from each entry in this column.

This step enhances the readability of the data and ensures that any subsequent text processing or analysis on this column does not get affected by these extraneous characters.


In [ ]:
# Remove square brackets from 'OorzaakVerloop' column
data['OorzaakVerloop'] = data['OorzaakVerloop'].str.replace('[', '', regex=False)
data['OorzaakVerloop'] = data['OorzaakVerloop'].str.replace(']', '', regex=False)

# Display the updated 'OorzaakVerloop' column
data['OorzaakVerloop'].head()


# Dataset Translation from Dutch to English

## Purpose
The dataset originally contains information in Dutch, which is not accessible or understandable to all potential users, especially those who are not proficient in Dutch. To make this dataset more universally accessible and easier to work with, we will translate it from Dutch to English.

## Scope
1. **Column Headers**: All column headers will be translated from Dutch to English to provide a clear understanding of the dataset's structure.
2. **Column Contents**: The textual contents of each column will also be translated. This step is crucial for columns containing descriptive information or categorical data.

## Method
We will use the `googletrans` library for this translation. This library provides a convenient way to access Google Translate's capabilities programmatically.

## Considerations
- The translation process can be time-consuming for large datasets.
- Automated translations may not always be perfect, and there could be nuances in the text that are not captured accurately.
- We will ensure to handle any translation errors and retain the original Dutch text where the translation is not feasible or produces unclear results.

By translating the dataset, we aim to enhance its accessibility and usability for a broader audience, facilitating better data understanding and analysis.


In [ ]:
from googletrans import Translator
import pandas as pd


# Create a Translator object
translator = Translator()

# Function to translate text
def translate_text(text, src_language='nl', dest_language='en'):
    """Translate the specified text from src_language to dest_language."""
    try:
        return translator.translate(text, src=src_language, dest=dest_language).text
    except Exception as e:
        print(f"Error while translating '{text}': {e}")
        return text  # Return original text if translation fails

# Translate column headers
translated_headers = [translate_text(header) for header in data.columns]

# Create a new DataFrame with translated headers
translated_data = pd.DataFrame(data.values, columns=translated_headers)

# Check the translated headers
print(translated_data.columns)



The translation given is not very accurate, so it will be done manually

In [ ]:

translated_headers = {
   'NLSitNummer': 'NL Site Number',
    'DatumFileBegin': 'File Start Date',
    'DatumFileEind': 'File End Date',
    'TijdFileBegin': 'File Start Time',
    'TijdFileEind': 'File End Time',
    'FileZwaarte': 'File Severity',
    'GemLengte': 'Average Length',
    'FileDuur': 'File Duration',
    'HectometerKop': 'Hectometer Head',
    'HectometerStaart': 'Hectometer Tail',
    'RouteLet': 'Route Letter',
    'RouteNum': 'Route Number',
    'RouteOms': 'Route Description',
    'hectometreringsrichting': 'Hectometering Direction',
    'KopWegvakVan': 'Head Road Section From',
    'KopWegvakNaar': 'Head Road Section To',
    'TrajVan': 'Trajectory From',
    'TrajNaar': 'Trajectory To',
    'OorzaakGronddetail': 'Cause Ground Detail',
    'OorzaakVerloop': 'Cause Progression',
    'OorzaakCodeVerloop': 'Cause Code Progression',
    'OorzaakCode': 'Cause Code',
    'Oorzaak_1': 'Cause 1',
    'Oorzaak_2': 'Cause 2',
    'Oorzaak_3': 'Cause 3',
    'Oorzaak_4': 'Cause 4',
  
}

# Replace the headers in the dataset
data.rename(columns=translated_headers, inplace=True)

# Save the translated dataset
data.to_csv('./translated_header_combined_data.csv', index=False)

now lets trasnlate the content of the column Hectometering Direction

In [ ]:
from googletrans import Translator, LANGUAGES
import pandas as pd

# Load your dataset
data = pd.read_csv('./translated_header_combined_data.csv')

# Create a Translator object
translator = Translator()

# Function to translate text and convert to lowercase
def translate_and_lowercase(text, src_language='nl', dest_language='en'):
    """Translate the specified text from src_language to dest_language and convert to lowercase."""
    try:
        translated_text = translator.translate(text, src=src_language, dest=dest_language).text
        return translated_text.lower()  # Convert to lowercase
    except Exception as e:
        print(f"Error while translating '{text}': {e}")
        return None  # Return None if translation fails

# Column name in your dataset that needs translation
column_to_translate = 'Hectometering Direction'

# Translate unique values in the column and convert to lowercase
unique_values = data[column_to_translate].unique()
translations = {val: translate_and_lowercase(val) for val in unique_values if isinstance(val, str)}

# Ensure all translations were successful
if None not in translations.values():
    # Apply the translations to the column
    data[column_to_translate] = data[column_to_translate].map(translations)

    # Save the translated dataset
    data.to_csv('./translated_hectometering_direction_combined_data.csv', index=False)
else:
    print("Translation failed for some values. Please check the errors.")
    
data.head(10)

The changes were succesfull.

next is Cause Ground Detail, additionaly I added some custom translation

In [ ]:
from googletrans import Translator
import pandas as pd

# Load your dataset
data = pd.read_csv('./translated_hectometering_direction_combined_data.csv')

# Create a Translator object
translator = Translator()

# Custom translations
custom_translations = {
    'File buiten spits (geen oorzaak gemeld)': 'Traffic jam outside rush hour (no cause reported)',
    'Spitsfile (geen oorzaak gemeld)': 'Rush hour traffic jam (no cause reported)',
    'Spitsfile (geen oorzaak gemeld) met gevonden werk in Spin': 'Rush hour traffic jam (no cause reported) with work found in Spin',
    'Afremmend verkeer als gevolg van kijkers naar  ongeval(len)': 'Traffic slowing down as a result of watching accident(s)',
    'Werkzaamheden (en daarna ongeval)': 'Work (and then accident)',

}

# Function to translate text and convert to lowercase
def translate_and_lowercase(text, src_language='nl', dest_language='en'):
    """Translate the specified text from src_language to dest_language and convert to lowercase."""
    # Use custom translation if available
    if text in custom_translations:
        return custom_translations[text].lower()

    # Otherwise, use the translator
    try:
        translated_text = translator.translate(text, src=src_language, dest=dest_language).text
        return translated_text.lower()  # Convert to lowercase
    except Exception as e:
        print(f"Error while translating '{text}': {e}")
        return None  # Return None if translation fails

# Column name in your dataset that needs translation
column_to_translate = 'Cause Ground Detail'

# Translate unique values in the column and convert to lowercase
unique_values = data[column_to_translate].unique()
translations = {val: translate_and_lowercase(val) for val in unique_values if isinstance(val, str)}

# Ensure all translations were successful
if None not in translations.values():
    # Apply the translations to the column
    data[column_to_translate] = data[column_to_translate].map(translations)

    # Save the translated dataset
    data.to_csv('./translated_cause_ground_detail_combined_data.csv', index=False)
else:
    print("Translation failed for some values. Please check the errors.")


data.head(10)


Yet again the changes are succesfull and now the column is more easily acceptable.

In [ ]:
import pandas as pd

# Load the dataset
file_path = './translated_cause_ground_detail_combined_data.csv'
data = pd.read_csv(file_path)


# Define a function to extract the 'VWM' part
def extract_vwm(value):
    if pd.isna(value) or not isinstance(value, str):
        return value  # Return the value as is if it's NaN or not a string
    # Extract 'VWM' followed by any number of digits
    vwm_part = pd.Series(value).str.extract(r'(VWM \d+)')[0]
    return vwm_part

# Apply the function to the 'Cause Progression' column
data['Cause Progression'] = data['Cause Progression'].apply(extract_vwm)


# Save the modified dataset
modified_file_path = './modified_cause_progression.csv'
data.to_csv(modified_file_path, index=False)


Here I apply filtering for routes only for Maarheeze and Eindhoven.

In [ ]:

file_path = './modified_cause_progression.csv'
data = pd.read_csv(file_path)

if not pd.api.types.is_numeric_dtype(data['Hectometer Head']):
    data['Hectometer Head'] = data['Hectometer Head'].str.replace(',', '.').astype(float)
if not pd.api.types.is_numeric_dtype(data['Hectometer Tail']):
    data['Hectometer Tail'] = data['Hectometer Tail'].str.replace(',', '.').astype(float)


data = data[
    (data['Hectometer Head'] < 182.0) &
    (data['Hectometer Tail'] > 155.0) &
    (data['Hectometering Direction'] == 'descending')
]


data.to_csv("./filtered_data", index=False)




modified_file_path = './filteredData.csv'
data.to_csv(modified_file_path, index=False)
data.info()

In [ ]:
import pandas as pd
from googletrans import Translator

# Load the dataset
file_path = './filteredData.csv'
data = pd.read_csv(file_path)

# Initialize the translator
translator = Translator()

# Function to translate text
def translate_text(text, src_language='nl', dest_language='en'):
    """Translate the specified text from src_language to dest_language."""
    try:
        return translator.translate(text, src=src_language, dest=dest_language).text
    except Exception as e:
        print(f"Error while translating '{text}': {e}")
        return text  # Return original text if translation fails

# List of columns to translate
columns_to_translate = ['Cause 1', 'Cause 2', 'Cause 3', 'Cause 4']

# Apply translation to each column
for col in columns_to_translate:
    # Ensure the column is of type string
    data[col] = data[col].astype(str)
    data[col] = data[col].apply(translate_text)

# Save the translated dataset
translated_file_path = './translated_causes.csv'
data.to_csv(translated_file_path, index=False)


The translation for some of the words didn't work so I will do it manually.

In [ ]:
import pandas as pd

# Load the dataset
file_path = './translated_causes.csv' 
data = pd.read_csv(file_path)

# Translation dictionary (Dutch to English)
# translation_dict = {
#     'Drukte': 'Crowd or Busyness',
#     'Ongeval': 'Accident',
#     'Hoge intensiteit': 'High Intensity',
#     'Aanleg en gepland onderhoud': 'Construction and Planned Maintenance',
#     'Incident': 'Incident',  
#     'Ongepland onderhoud': 'Unplanned Maintenance',
#     'Geen oorzaak gemeld' : 'No cause reported'
# }

translation_dict = {
    '(Zeer extreme) weersomstandigheden': '(Very extreme) weather conditions',
    'Acties of demonstraties': 'Actions or demonstrations',
    'Afremmend verkeer als gevolg van kijkers naar  ongeval(len)': 'Traffic slowing down due to onlookers at accidents',
    'Betoging': 'Demonstration',
    'Brandend(e) voertuig(en)': 'Burning vehicle(s)',
    'Dieren op de weg': 'Animals on the road',
    'Eerder pechgeval': 'Earlier breakdown',
    'Eerder(e) ongeval(len)': 'Previous accident(s)',
    'Evenement': 'Event',
    'File buiten spits (geen oorzaak gemeld)': 'Traffic jam outside rush hour (no cause reported)',
    'Gekanteld(e) vrachtwagen(s)': 'Tilted truck(s)',
    'Gevaarlijke situatie': 'Dangerous situation',
    'Incident (gestrand voertuig)': 'Incident (stranded vehicle)',
    'Incident (obstakel op de weg)': 'Incident (obstacle on the road)',
    'Incident (opruimwerkzaamheden)': 'Incident (cleanup work)',
    'Incident op aansluitende weg': 'Incident on a connecting road',
    'Kijkfile': 'Rubbernecking traffic jam',
    'Lading op de weg': 'Load on the road',
    'Langza(a)m(e) voertuig(en)': 'Slow-moving vehicle(s)',
    'Ongeval op aansluitende weg': 'Accident on a connecting road',
    'Onwelwording': 'Sudden illness',
    'Opgehoogde werkzaamheden': 'Elevated work',
    'Opruimwerkzaamheden': 'Cleanup work',
    'Politiecontrole': 'Police control',
    'Spitsfile (geen oorzaak gemeld)': 'Rush hour traffic jam (no cause reported)',
    'Spoedreparatie': 'Emergency repair',
    'Voorwerpen op de weg': 'Objects on the road',
    'Wegwerkzaamheden': 'Road work',
    'Werkzaamheden': 'Activities',
    'Werkzaamheden gevonden door WiFi-script': 'Work detected by WiFi script',

    'Accident': 'Accident',
    'Accident (len)': 'Accident',
    'Accident elsewhere': 'Accident elsewhere',
    'Accident handling': 'Accident handling',
    'Accident on the connecting road': 'Accident on the connecting road',
    'Accident truck': 'Accident truck',
    'Accident with truck (s)': 'Accident with truck(s)',
    'Actions or demonstrations': 'Actions or demonstrations',
    'Burning (s)': 'Burning vehicle(s)',
    'Clean -up work': 'Clean-up work',
    'Clearance work': 'Clearance work',
    'Construction and Planned Maintenance': 'Construction and Planned Maintenance',
    'Construction and planned maintenance': 'Construction and planned maintenance',
    'Crowd or Busyness': 'Crowd or Busyness',
    'Damaged guide rail': 'Damaged guide rail',
    'Defect voertuig': 'Defective vehicle',
    'Defect(e) voertuig(en)': 'Defective vehicle(s)',
    'Defecte vrachtwagen': 'Defective truck',
    'Defecte vrachtwagen(s)': 'Defective truck(s)',
    'Defective truck': 'Defective truck',
    'Defective truck (s)': 'Defective truck(s)',
    'Defective vehicle (s)': 'Defective vehicle(s)',
    'Demonstration': 'Demonstration',
    'Event': 'Event',
    'Extra traffic due to traffic that follows a diversion': 'Extra traffic due to diversion',
    'Failure': 'Failure',
    'Fallen load (s)': 'Fallen load(s)',
    'Fire near the road': 'Fire near the road',
    'Fraamy traffic due to viewers to accident (len)': 'Traffic slowing due to onlookers at accidents',
    'Freezing of wet roads': 'Freezing of wet roads',
    'High Intensity': 'High Intensity',
    'High intensity': 'High Intensity',
    'Incident': 'Incident',
    'Incident (clean -up work)': 'Incident (cleanup work)',
    'Incident (obstacle on the road)': 'Incident (obstacle on the road)',
    'Incident (s)': 'Incident(s)',
    'Incident elders': 'Incident elsewhere',
    'Incident on a subsequent road': 'Incident on a subsequent road',
    'Incident(en)': 'Incidents',
    'Infrastructure damage': 'Infrastructure damage',
    'Load on the road': 'Load on the road',
    'Long -term road works': 'Long-term road works',
    'No cause reported': 'No cause reported',
    'Objects on the road': 'Objects on the road',
    'People or animals on the road': 'People or animals on the road',
    'Pointed file (not reported)': 'Pointed traffic jam (not reported)',
    'Police check': 'Police check',
    'Police control post': 'Police control post',
    'Police investigation': 'Police investigation',
    'Previous accident on a subsequent road': 'Previous accident on a subsequent road',
    'Previously accident (len)': 'Previously accident(s)',
    'Raised work': 'Raised work',
    'Rather breakdown': 'Earlier breakdown',
    'Road work': 'Road work',
    'Rush': 'Rush',
    'Slippery road': 'Slippery road',
    'Spitsfile (geen oorzaak gemeld) met gevonden werk in Spin': 'Rush hour traffic jam (no cause reported) with found work in spin',
    'Spitsfile (no cause reported) with found work in spin': 'Rush hour traffic jam (no cause reported) with found work in spin',
    'There is clutter on the road': 'Clutter on the road',
    'Tilted truck (s)': 'Tilted truck(s)',
    'Unplanned Maintenance': 'Unplanned Maintenance',
    'Unplanned maintenance': 'Unplanned maintenance',
    'Unqualification': 'Unqualification',
    'Vehicle defective': 'Defective vehicle',
    'Defect voertuig': 'Defective Vehicle',
    'Defecte vrachtwagen': 'Defective Truck',
    'File buiten spits (geen oorzaak gemeld)': 'Off-Peak Traffic Jam (No Cause Reported)',
    'Ongeval': 'Accident',
    'Ongeval op aansluitende weg': 'Accident on Connecting Road',
    'Onwelwording': 'Sudden Illness',
    'Opgehoogde werkzaamheden': 'Raised Construction Work',
    'Spitsfile (geen oorzaak gemeld)': 'Rush Hour Traffic Jam (No Cause Reported)',
    'Spoedreparatie': 'Emergency Repair',
    'Verkeer ingesloten tussen afrit en incident': 'Traffic Trapped Between Exit and Incident',
    'Voorwerpen op de weg': 'Objects on the Road',
    'Wegwerkzaamheden': 'Road Works',
    'Zeer extreme weersomstandigheden': 'Extremely Severe Weather Conditions',
    'Accident with truck (s)': 'Accident with Truck(s)',
    'Construction and Planned Maintenance': 'Construction and Planned Maintenance',
    'Crowd or Busyness': 'Crowd or Busyness',
    'High Intensity': 'High Intensity',
    'Incident': 'Incident',
    'No cause reported': 'No Cause Reported',
    'Unplanned Maintenance': 'Unplanned Maintenance'

}




# List of columns to translate
columns_to_translate = ['Cause 1', 'Cause 2', 'Cause 3', 'Cause 4']

# Apply manual translation to each column
for col in columns_to_translate:
    data[col] = data[col].map(translation_dict).fillna(data[col])

# Save the translated dataset
translated_file_path = './translated_causes.csv'
data.to_csv(translated_file_path, index=False)

print("Translation complete. Translated data saved to:", translated_file_path)


and finaly I am going to drop the useless columns like Couse Code Progression and Cause Code

In [ ]:
file_path = './translated_causes.csv' 
data = pd.read_csv(file_path)
data.drop(columns=["Cause Code Progression", "Cause Code"], inplace=True)

final_data = './final.csv'
data.to_csv(final_data, index=False)


The needed road is A2 so I will also filter on that road.

In [ ]:
file_path = './final.csv'
data = pd.read_csv(file_path)
data = data[data["Route Description"] == "A2"]

data.head





final_data = './A2data.csv'
data.to_csv(final_data, index=False)





The data here is from Maarheeze to Eindhoven so I will add a new column that indicates that information. The column will store the value "M-E"

In [ ]:
file_path = './A2data.csv'
data = pd.read_csv(file_path)

data['route'] = "M-E"

final_data = './dataWithColumn.csv'

data.to_csv(final_data, index=False)


Now I will filter the data for Eindhoven to Den bosh and apply the same logic

In [ ]:

file_path = './modified_cause_progression.csv'
data = pd.read_csv(file_path)

if not pd.api.types.is_numeric_dtype(data['Hectometer Head']):
    data['Hectometer Head'] = data['Hectometer Head'].str.replace(',', '.').astype(float)
if not pd.api.types.is_numeric_dtype(data['Hectometer Tail']):
    data['Hectometer Tail'] = data['Hectometer Tail'].str.replace(',', '.').astype(float)


data = data[
    (data['Hectometer Head'] < 154.8) &
    (data['Hectometer Tail'] > 121.7) &
    (data['Hectometering Direction'] == 'descending')
]


data.to_csv("./dataEindhovenDenBosch.csv", index=False)




# modified_file_path = './filteredData.csv'
# data.to_csv(modified_file_path, index=False)
# data.info()

Now let's translate the columns

In [ ]:
import pandas as pd

# Load the dataset
file_path = './dataEindhovenDenBosch.csv' 
data = pd.read_csv(file_path)



translation_dict = {
    'Aanleg en gepland onderhoud': 'Construction and Planned Maintenance',
    'Acties of demonstraties': 'Actions or Demonstrations',
    'Afremmend verkeer als gevolg van kijkers naar ongeval(len)': 'Traffic Slowing Down Due to Onlookers at Accidents',
    'Andere wegbeheerder': 'Other Road Manager',
    'Bermbrand': 'Roadside Fire',
    'Betoging': 'Protest',
    'Brand in de buurt van de weg': 'Fire Near the Road',
    'Brandend(e) voertuig(en)': 'Burning Vehicle(s)',
    'Cultureel evenement': 'Cultural Event',
    'Defect voertuig': 'Defective Vehicle',
    'Defect(e) voertuig(en)': 'Defective Vehicle(s)',
    'Defecte bus(sen)': 'Defective Bus(es)',
    'Defecte vrachtwagen': 'Defective Truck',
    'Defecte vrachtwagen(s)': 'Defective Truck(s)',
    'Drukte': 'Crowd or Busyness',
    'Eerder pechgeval': 'Earlier Breakdown',
    'Eerder(e) ongeval(len)': 'Previous Accident(s)',
    'Er ligt rommel op de weg': 'Clutter on the Road',
    'Evenement': 'Event',
    'Extra verkeersdrukte door verkeer dat een omleiding volgt': 'Extra Traffic Due to Traffic Following a Diversion',
    'Falen Infra': 'Infrastructure Failure',
    'Festival': 'Festival',
    'File buiten spits (geen oorzaak gemeld)': 'Off-Peak Traffic Jam (No Cause Reported)',
    'Gekanteld(e) voertuig(en)': 'Tilted Vehicle(s)',
    'Gekantelde aanhangwagen': 'Tilted Trailer',
    'Gevaarlijke situatie': 'Dangerous Situation',
    'Hoge intensiteit': 'High Intensity',
    'Incident (gestrand voertuig)': 'Incident (Stranded Vehicle)',
    'Incident (obstakel op de weg)': 'Incident (Obstacle on the Road)',
    'Incident (opruimwerkzaamheden)': 'Incident (Cleanup Operations)',
    'Incident elders': 'Incident Elsewhere',
    'Incident op aansluitende weg': 'Incident on Connecting Road',
    'Kijkfile': 'Rubbernecking Traffic Jam',
    'Lading op de weg': 'Load on the Road',
    'Langdurige wegwerkzaamheden': 'Long-Term Roadworks',
    'Langza(a)m(e) voertuig(en)': 'Slow-Moving Vehicle(s)',
    'Olie op het wegdek': 'Oil on the Road Surface',
    'Ongeval op provinciale weg': 'Accident on Provincial Road',
    'Ongevalsafhandeling': 'Accident Handling',
    'Ongevalsonderzoek': 'Accident Investigation',
    'Opruimingswerkzaamheden': 'Cleanup Operations',
    'Overige oorzaken': 'Other Causes',
    'Politieactie': 'Police Action',
    'Politieonderzoek': 'Police Investigation',
    'Provinciale weg': 'Provincial Road',
    'Recreatie- en vakantieverkeer': 'Recreational and Holiday Traffic',
    'Slecht wegdek': 'Poor Road Surface',
    'Vakantieverkeer': 'Holiday Traffic',
    'Vloeistof op de weg': 'Liquid on the Road',
    'Vloeistof op het wegdek': 'Liquid on the Road Surface',
    'Water op de weg': 'Water on the Road',
    'Wegdek in slechte toestand': 'Road Surface in Poor Condition',
    'Werkzaamheden elders': 'Works Elsewhere',
    'Werkzaamheden op aansluitende weg': 'Works on Connecting Road',
    'Geen oorzaak gemeld': 'No Cause Reported',
    'Incident': 'Incident',
    'Ongepland onderhoud': 'Unplanned Maintenance',
    'Ongeval': 'Accident',
    'Ongeval elders': 'Accident Elsewhere',
    'Ongeval met vrachtwagen(s)': 'Accident with Truck(s)',
    'Ongeval op aansluitende weg': 'Accident on Connecting Road',
    'Ongeval vrachtwagen': 'Truck Accident',
    'Ongeval(len)': 'Accidents',
    'Spitsfile (geen oorzaak gemeld)': 'Rush Hour Traffic Jam (No Cause Reported)',
    'Spitsfile (geen oorzaak gemeld) met gevonden werk in Spin': 'Rush Hour Traffic Jam (No Cause Reported) with Found Work in Spin',
    'Spoedreparatie': 'Emergency Repair',
    'Voorwerpen op de weg': 'Objects on the Road',
    'Wegwerkzaamheden': 'Road Works',
    'Werkzaamheden': 'Works',
    'Werkzaamheden gevonden door WiFi-script': 'Works Detected by WiFi Script'
}




# List of columns to translate
columns_to_translate = ['Cause 1', 'Cause 2', 'Cause 3', 'Cause 4']

# Apply manual translation to each column
for col in columns_to_translate:
    data[col] = data[col].map(translation_dict).fillna(data[col])

# Save the translated dataset
translated_file_path = './translatedDataEindhovenDenBosch.csv'
data.to_csv(translated_file_path, index=False)

print("Translation complete. Translated data saved to:", translated_file_path)


now I am going to drop the useless columns

In [ ]:
file_path = './translatedDataEindhovenDenBosch.csv' 
data = pd.read_csv(file_path)
data.drop(columns=["Cause Code Progression", "Cause Code"], inplace=True)

final_data = './droppedcolumnsEindhivenDenBosch.csv'
data.to_csv(final_data, index=False)

The highway from Eindhoven to Den Bosch is number A2 so I am going to filter the data on that road.

In [ ]:
file_path = './droppedcolumnsEindhivenDenBosch.csv'
data = pd.read_csv(file_path)
data = data[data["Route Description"] == "A2"]

data.head





final_data = './A2dataEindhovenDenBosch.csv'
data.to_csv(final_data, index=False)





Now I am going to add a column to specify that the data here is from Eindhoven to Den Bosch

In [ ]:
file_path = './A2dataEindhovenDenBosch.csv'
data = pd.read_csv(file_path)

data['route'] = "E-D"

final_data = './dataEindhovenDenBosch.csv'

data.to_csv(final_data, index=False)


Now I will filter the data for Eindhoven to Arnhem and apply the same logic

In [ ]:

file_path = './modified_cause_progression.csv'
data = pd.read_csv(file_path)

if not pd.api.types.is_numeric_dtype(data['Hectometer Head']):
    data['Hectometer Head'] = data['Hectometer Head'].str.replace(',', '.').astype(float)
if not pd.api.types.is_numeric_dtype(data['Hectometer Tail']):
    data['Hectometer Tail'] = data['Hectometer Tail'].str.replace(',', '.').astype(float)




data = data[
    (data['Hectometer Head'] < 162.2) &
    (data['Hectometer Tail'] > 154.8) &
    (data['Hectometering Direction'] == 'ascending')
]


data.to_csv("./dataEindhovenArnhem.csv", index=False)




# modified_file_path = './filteredData.csv'
# data.to_csv(modified_file_path, index=False)
# data.info()

Now I will translate the data.

In [ ]:
import pandas as pd

# Load the dataset
file_path = './dataEindhovenArnhem.csv' 
data = pd.read_csv(file_path)



translation_dict = {
    'Aanleg en gepland onderhoud': 'Construction and Planned Maintenance',
    'Acties of demonstraties': 'Actions or Demonstrations',
    'Afremmend verkeer als gevolg van kijkers naar ongeval(len)': 'Traffic Slowing Down Due to Onlookers at Accidents',
    'Andere wegbeheerder': 'Other Road Manager',
    'Bermbrand': 'Roadside Fire',
    'Betoging': 'Protest',
    'Brand in de buurt van de weg': 'Fire Near the Road',
    'Brandend(e) voertuig(en)': 'Burning Vehicle(s)',
    'Cultureel evenement': 'Cultural Event',
    'Defect voertuig': 'Defective Vehicle',
    'Defect(e) voertuig(en)': 'Defective Vehicle(s)',
    'Defecte bus(sen)': 'Defective Bus(es)',
    'Defecte vrachtwagen': 'Defective Truck',
    'Defecte vrachtwagen(s)': 'Defective Truck(s)',
    'Drukte': 'Crowd or Busyness',
    'Eerder pechgeval': 'Earlier Breakdown',
    'Eerder(e) ongeval(len)': 'Previous Accident(s)',
    'Er ligt rommel op de weg': 'Clutter on the Road',
    'Evenement': 'Event',
    'Extra verkeersdrukte door verkeer dat een omleiding volgt': 'Extra Traffic Due to Traffic Following a Diversion',
    'Falen Infra': 'Infrastructure Failure',
    'Festival': 'Festival',
    'File buiten spits (geen oorzaak gemeld)': 'Off-Peak Traffic Jam (No Cause Reported)',
    'Gekanteld(e) voertuig(en)': 'Tilted Vehicle(s)',
    'Gekantelde aanhangwagen': 'Tilted Trailer',
    'Gevaarlijke situatie': 'Dangerous Situation',
    'Hoge intensiteit': 'High Intensity',
    'Incident (gestrand voertuig)': 'Incident (Stranded Vehicle)',
    'Incident (obstakel op de weg)': 'Incident (Obstacle on the Road)',
    'Incident (opruimwerkzaamheden)': 'Incident (Cleanup Operations)',
    'Incident elders': 'Incident Elsewhere',
    'Incident op aansluitende weg': 'Incident on Connecting Road',
    'Kijkfile': 'Rubbernecking Traffic Jam',
    'Lading op de weg': 'Load on the Road',
    'Langdurige wegwerkzaamheden': 'Long-Term Roadworks',
    'Langza(a)m(e) voertuig(en)': 'Slow-Moving Vehicle(s)',
    'Olie op het wegdek': 'Oil on the Road Surface',
    'Ongeval op provinciale weg': 'Accident on Provincial Road',
    'Ongevalsafhandeling': 'Accident Handling',
    'Ongevalsonderzoek': 'Accident Investigation',
    'Opruimingswerkzaamheden': 'Cleanup Operations',
    'Overige oorzaken': 'Other Causes',
    'Politieactie': 'Police Action',
    'Politieonderzoek': 'Police Investigation',
    'Provinciale weg': 'Provincial Road',
    'Recreatie- en vakantieverkeer': 'Recreational and Holiday Traffic',
    'Slecht wegdek': 'Poor Road Surface',
    'Vakantieverkeer': 'Holiday Traffic',
    'Vloeistof op de weg': 'Liquid on the Road',
    'Vloeistof op het wegdek': 'Liquid on the Road Surface',
    'Water op de weg': 'Water on the Road',
    'Wegdek in slechte toestand': 'Road Surface in Poor Condition',
    'Werkzaamheden elders': 'Works Elsewhere',
    'Werkzaamheden op aansluitende weg': 'Works on Connecting Road',
    'Geen oorzaak gemeld': 'No Cause Reported',
    'Incident': 'Incident',
    'Ongepland onderhoud': 'Unplanned Maintenance',
    'Ongeval': 'Accident',
    'Ongeval elders': 'Accident Elsewhere',
    'Ongeval met vrachtwagen(s)': 'Accident with Truck(s)',
    'Ongeval op aansluitende weg': 'Accident on Connecting Road',
    'Ongeval vrachtwagen': 'Truck Accident',
    'Ongeval(len)': 'Accidents',
    'Spitsfile (geen oorzaak gemeld)': 'Rush Hour Traffic Jam (No Cause Reported)',
    'Spitsfile (geen oorzaak gemeld) met gevonden werk in Spin': 'Rush Hour Traffic Jam (No Cause Reported) with Found Work in Spin',
    'Spoedreparatie': 'Emergency Repair',
    'Voorwerpen op de weg': 'Objects on the Road',
    'Wegwerkzaamheden': 'Road Works',
    'Werkzaamheden': 'Works',
    'Werkzaamheden gevonden door WiFi-script': 'Works Detected by WiFi Script'
}




# List of columns to translate
columns_to_translate = ['Cause 1', 'Cause 2', 'Cause 3', 'Cause 4']

# Apply manual translation to each column
for col in columns_to_translate:
    data[col] = data[col].map(translation_dict).fillna(data[col])

# Save the translated dataset
translated_file_path = './translatedDataEindhovenArnhem.csv'
data.to_csv(translated_file_path, index=False)

print("Translation complete. Translated data saved to:", translated_file_path)


now I am going to drop the useless columns

In [ ]:
file_path = './translatedDataEindhovenArnhem.csv' 
data = pd.read_csv(file_path)
data.drop(columns=["Cause Code Progression", "Cause Code"], inplace=True)

final_data = './droppedcolumnsEindhivenArnhem.csv'
data.to_csv(final_data, index=False)

The highway from Eindhoven to Arnhem is number A50 so I am going to filter the data on that road.

In [ ]:
file_path = './droppedcolumnsEindhivenArnhem.csv'
data = pd.read_csv(file_path)
data = data[data["Route Description"] == "A50"]

data.head





final_data = './A50dataEindhovenArnhem.csv'
data.to_csv(final_data, index=False)





Now I am going to add a column to specify that the data here is from Eindhoven to Arnhem

In [ ]:
file_path = './A50dataEindhovenArnhem.csv'
data = pd.read_csv(file_path)

data['route'] = "E-A"

final_data = './dataEindhovenArnhemColumn.csv'

data.to_csv(final_data, index=False)
